In [2]:
!pip install tf_slim

     |████████████████████████████████| 358kB 3.0MB/s 


In [3]:
import os
import tensorflow as tf
import numpy as np
import cv2
import random
import scipy.misc
import tf_slim as slim
import keras

from IPython import display

from PIL import Image
import matplotlib.pyplot as plt

In [4]:
latent_dim = 100
HEIGHT, WIDTH, CHANNEL = 64, 64, 3
BATCH_SIZE = 128
EPOCH = 10000

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
''''import pathlib
data_dir = pathlib.Path("gdrive/MyDrive/projects/images/")''''

In [ ]:
'''image_count = len(list(data_dir.rglob('*.png')))
print(image_count)'''

6817


# **Loss**

In [6]:
def loss_generator(logit_fake):
    """
    loss function for generator.
    """
    return tf.math.negative(tf.math.reduce_mean(tf.math.log(logit_fake)))


def loss_discriminator(logit_real, logit_fake):
    """
    loss function for discriminator.
    """
    loss_real = tf.math.negative(tf.math.reduce_mean(tf.math.log(logit_real)))
    loss_fake = tf.math.negative(tf.math.reduce_mean(tf.math.log(1. - logit_fake)))
    return loss_real + loss_fake

# **Generator**

In [7]:
def Generator():
  generator_input = keras.Input(shape = (latent_dim,))

  x = keras.layers.Dense(16384, activation = "linear")(generator_input)
  x = keras.layers.Reshape((4,4,1024))(x)

  x = keras.layers.Conv2DTranspose(512, 4, strides = 2, padding='same', activation = 'relu')(x)
  x = keras.layers.Conv2DTranspose(256, 4, strides = 2, padding = 'same', activation = 'relu')(x)
  x = keras.layers.Conv2DTranspose(128, 4, strides = 2, padding = 'same', activation = 'relu')(x)
  x = keras.layers.Conv2DTranspose(3, 4, strides = 2, padding = 'same', activation = 'tanh')(x)

  model = keras.models.Model(inputs = generator_input, outputs = x)
  return model

# **Discriminator**

In [8]:
def Discriminator():
  discriminator_input = keras.layers.Input(shape = (HEIGHT, WIDTH, CHANNEL))
  x = keras.layers.Conv2D(128, 4, strides = 2, padding = "same")(discriminator_input)
  x = keras.layers.LeakyReLU(0.2)(x)
  x = keras.layers.Conv2D(256, 4, strides = 2, padding = "same")(x)
  x = keras.layers.LeakyReLU(0.2)(x)
  x = keras.layers.Conv2D(512, 4, strides = 2, padding = "same")(x)
  x = keras.layers.LeakyReLU()(x)
  x = keras.layers.Conv2D(1024, 4, strides = 2, padding = "same")(x)
  x = keras.layers.LeakyReLU()(x)

  x = keras.layers.Reshape((1,16384), input_shape=(4,1024))(x)

  x = keras.layers.Dense(1, activation = 'linear')(x)
  x = keras.layers.Dense(5, activation = 'linear')(x)
  x = keras.layers.Dense(2, activation = 'linear')(x)

  model = keras.models.Model(inputs = discriminator_input, outputs = x)
  return model


# **Create gen and dis**

In [9]:
def create_generator():
    #creates generator, its optimizer (Adam) and checkpoint.

    generator = Generator()
    generator_optimizer = Adam(lr=0.002, beta_1=0.5)
    generator_checkpoint = tf.train.Checkpoint(optimizer=generator_optimizer, model=generator)

    return generator, generator_optimizer, generator_checkpoint

In [10]:
def create_discriminator():
    #creates discriminator, its optimizer (Adam) and checkpoint.

    discriminator = Discriminator()
    discriminator_optimizer = Adam(lr=0.001, beta_1=0.5)
    discriminator_checkpoint = tf.train.Checkpoint(optimizer=discriminator_optimizer, model=discriminator)

    return discriminator, discriminator_optimizer, discriminator_checkpoint

# **Save images function**

In [11]:
def generate_print_save(model, input, epoch):
    """
    generates, prints and saves 9 images for a given input noise of shape [9, noise_size],
    where noise_size is globally defined.
    """
    prediction = model(input, training=False)
    fig = plt.figure(figsize=(10, 10))

    for i in range(9):
        plt.subplot(3, 3, i + 1)
        plt.imshow(prediction[i])
        plt.axis('off')

    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch+1))
    plt.show()

In [12]:

def read_images(data_dir):
    """
    reads images from data_dir into a numpy array of shape
    (number of images, height, width, number of channels (=3 for rgb)).
    pixels of each image are normalized (in the interval [-1, 1]).
    """
    images = list()
    for file_name in data_dir:
        image = Image.open(file_name)
        image = np.asarray(image)
        images.append(image)

    return (np.asarray(images) - 127.5) / 127.5
    


def print_images(images):
    """
    prints first 9 images (in 3 columns and 3 rows) from images numpy array.
    """
    for i in range(9):
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i])
        plt.axis('off')

    plt.show()

### **Training**

In [13]:
def train_DCGAN(train_set, batch_size, epochs):
    """
    trains the globally defined generator and discriminator.
    """
    # train set size
    train_size = train_set.shape[0]

    # the size of noise vector is given globally
    noise_size = latent_dim

    # number of mini batches
    m = train_size // batch_size

    # partition the training set to mini batches
    train_batches = np.split(train_set, [k * batch_size for k in range(1, m)])


    # prefixes for the checkpoints of the generator and discriminator
    generator_prefix = os.path.join('./generator', 'ckpt')
    discriminator_prefix = os.path.join('./discriminator', 'ckpt')

    # lists to record costs of the generator and discriminator at every 10 epochs
    generator_costs = list()
    discriminator_costs = list()

    for epoch in range(epochs):

        # initiate costs of the generator and discriminator at the current epoch to zeros
        generator_cost = 0
        discriminator_cost = 0

        for batch in train_batches:
            # random noise for the current mini batch
            noise = tf.random.normal([batch_size, noise_size])

            # watch trainable variables for the loss functions of the generator and discriminator
            with tf.GradientTape() as generator_tape, tf.GradientTape() as discriminator_tape:
                fake_images = generator(noise, training=True) # the generator generates fake images

                # the discriminator computes logits (probabilites) of images for being real
                logit_fake = discriminator(fake_images, training=True)
                logit_real = discriminator(batch, training=True)

                # loss functions for the generator and discriminator
                generator_loss = loss_generator(logit_fake)
                discriminator_loss = loss_discriminator(logit_real, logit_fake)

            # compute gradients and perform one step gradient descend
            generator_Grads = generator_tape.gradient(generator_loss, generator.trainable_variables)
            generator_optimizer.apply_gradients(zip(generator_Grads, generator.trainable_variables))
            discriminator_Grads = discriminator_tape.gradient(discriminator_loss, discriminator.trainable_variables)
            discriminator_optimizer.apply_gradients(zip(discriminator_Grads, discriminator.trainable_variables))

            # record costs of the generator and discriminator at every 10 epochs
            if (epoch + 1) % 10 == 0:
                fake_images = generator(noise, training=False)
                logit_fake = discriminator(fake_images, training=False)
                logit_real = discriminator(batch, training=False)
                generator_cost += loss_generator(logit_fake).numpy() / m
                discriminator_cost += loss_discriminator(logit_real, logit_fake).numpy() / m

        # save checkpoints at every 10 epochs
        # also print and save 9 randomly generated images at every 10 epochs
        if (epoch + 1) % 10 == 0:
            generator_checkpoint.save(file_prefix=generator_prefix)
            discriminator_checkpoint.save(file_prefix=discriminator_prefix)

            generator_costs.append(generator_cost)
            discriminator_costs.append(discriminator_cost)

            display.clear_output(wait=True)
            print('Epoch: {}'.format(epoch+1))
            print('Generator loss: {}'.format(generator_loss))
            print('Discriminator loss: {}'.format(discriminator_loss))
            noise = tf.random.normal([9, noise_size])
            generate_print_save(generator, noise, epoch)

    # plot the learning curves of the generator and discriminator
    plt.plot(np.squeeze(generator_costs))
    plt.plot(np.squeeze(discriminator_costs))
    plt.show()

# **Download images**

In [ ]:
import pathlib
data_dir = pathlib.Path("gdrive/MyDrive/projects/images/pop")
train_set = read_images(data_dir.rglob('*.png'))

In [ ]:
# let us look at few images from the dataset
print_images(images=train_set)

In [ ]:
generator, generator_optimizer, generator_checkpoint = create_generator()
discriminator, discriminator_optimizer, discriminator_checkpoint = create_discriminator()

# train generator and discriminator
train_DCGAN(train_set, batch_size=BATCH_SIZE, epochs=EPOCH)